In [11]:
from PIL import Image
from numpy import*
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder
from scipy.misc import imread, imsave

In [12]:
## Function to create a grid from a png image of the map. This must be a masked image from inkscape (black boxes 
## over everything that is not part of a valid path)

def createGrid(imageFile, outputFile, shrinkFactor):
    ## Defining 0 as obstacle, 1 as land 

    img = Image.open(imageFile)

    basewidth = int(img.size[0]/shrinkFactor)
    wpercent = (basewidth/float(img.size[0]))
    
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    img.save(outputFile) 
    
    temp=asarray(Image.open(outputFile))
    new_A=empty((temp.shape[0],temp.shape[1]),None)   

    for y in range(len(new_A)):
        for x in range(len(new_A[y])):
            if temp[y][x][3]==255:
                new_A[y][x]=0
            else:
                new_A[y][x]=1
    
    
                
    grid = Grid(matrix=new_A)
    
    return grid
    

    

In [13]:
## Create function to generate a path from one direction to another 
## Uses coordinates from SHRUNK image (shrinkFactor must be same as that used in the grid creation method)

def generatePath(startX, startY, endX, endY, grid, shrinkFactor):
    
    start = grid.node(startX, startY)
    end = grid.node(endX, endY)
    
    finder = AStarFinder()
    path, runs = finder.find_path(start, end, grid)
    
    # Create a path that only considers turns 
    simplePath=[(startX * shrinkFactor, startY * shrinkFactor)]
    
    lastX = startX
    lastY = startY
    
    if path:
        for i in range(1, len(path)):
            x = path[i][0]
            y = path[i][1]
        
            if x is not lastX and y is not lastY:
                lastX = x
                lastY = y
                simplePath.append((path[i-1][0] * shrinkFactor, path[i-1][1] * shrinkFactor))

        simplePath.append((path[len(path)-1][0] * shrinkFactor, path[len(path)-1][1] * shrinkFactor))

        d_string = ""


        for x, y in simplePath[1:]:
            move = " " + "L " + str(x) + " " + str(y)
            d_string += move

    else:
        d_string = ""
        
        print("No path found", startX, startY, endX, endY)
  
    
    
    return d_string, len(path) * shrinkFactor
        


In [16]:
## Define pixel locations of item categories

nodes = [
         {"type": "entrance", "x": 148, "y": 223, "key": 0},
         {"type": "fruit", "x": 85, "y": 66, "key": 1}, 
         {"type": "bread", "x": 26, "y": 37, "key": 2}, 
         {"type": "alcohol", "x": 172, "y": 38, "key": 3},
         {"type": "produce", "x": 264, "y": 76, "key": 4},
        ]

## later probably add all keywords associated with each category 

In [15]:
## Write function to generate path and weight from each node to all others
## Format: matrix[node1][node2] is the path FROM node1 TO node2

def generatePathMatrix(nodes, inputImg, outputImg, shrinkFactor):
    path_matrix = [[None for i in range(len(nodes))] for j in range(len(nodes))]
    cost_matrix = [[None for i in range(len(nodes))] for j in range(len(nodes))]
    
    grid = createGrid(inputImg, outputImg, shrinkFactor)

    
    for row in range(len(path_matrix)):
        for col in range(len(path_matrix)):
            if row == col:
                path_matrix[row][col] = ""
                cost_matrix[row][col] = 0
            else:
                grid = createGrid(inputImg, outputImg, shrinkFactor)
                path, distance = generatePath(nodes[row]["x"], nodes[row]["y"], nodes[col]["x"], nodes[col]["y"], grid, shrinkFactor) 
                path_matrix[row][col] = path
                cost_matrix[row][col] = distance
                
    return path_matrix, cost_matrix
path_matrix, cost_matrix = generatePathMatrix(nodes, "Map3Mask.png", "SmallMap3Mask.png", 10)


In [8]:
import json

data = {"paths": path_matrix, "costs": cost_matrix, "nodes": nodes}

with open('store1.json', 'w') as outfile:
    json.dump(data, outfile, sort_keys=True, indent=4)